In [80]:
import pandas as pd
import re, string, unicodedata, nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

df = pd.read_csv('../../data/bases_disciplina_2/tweets_classificados.csv', encoding='utf-8')
df.head()

,id,data_tweet,texto,sentimento
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",Neutro
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",Neutro
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,Neutro
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,Negativo


# ToDo 1

Altere as funções de tratamento de texto apresentadas em sala para que elas façam a remoção de links também.

Crie uma nova coluna chamada texto_tratado que conterá o resultado da aplicação das funções.

## Funções existentes

In [81]:
# Remove acentos
def normalize_accents(text):
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')

# Remove pontuação
def remove_punctuaction(text):
    punctiations = string.punctuation
    table = str.maketrans({key: " " for key in punctiations})
    text = text.translate(table)
    return text

# Remove URLs
def remove_links(text):
    return re.sub(r"\S*https?:\S*", "", text)

# Converte para minusculua, aplica funções de normalização e retira espaços em branco adicionais
def normalize_str(text):
    text = text.lower()
    text = remove_links(text)
    text = remove_punctuaction(text)
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +")," ", text)
    return ' '.join([w for w in text.split()])

# Função completa de tokenização com exclusão de stopwords e verificação de dtype
def tokenizer(text):
    stop_words = nltk.corpus.stopwords.words('english')

    if isinstance(text, str):
        text = normalize_str(text)
        text = ''.join([w for w in text if not w.isdigit()])
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words]
        text = [y for y in text if len(y) >= 2]
        return ' '.join([t for t in text])
    else:
        print('Not a string')

In [82]:
tokenizer('Meu link é asdasd(https://twitterr34321)sadasd')

'meu link'

# ToDo 2

Ao fazer a remoção de links, percebemos que algumas linhas da coluna texto_tratado possuem valores faltantes. Entretanto, o Python trata eles como ''(str) e nao como Null. Assim, um simples dropna nao resolve o problema.

Encontre uma forma de remover tais elementos. Dica: use o índice das linhas cujos elementos da coluna texto_tratado seja nulo.

In [83]:
df['texto_norm'] = df['texto'].apply(tokenizer)

In [84]:
df.loc[df['texto_norm'] == '', 'texto_norm'] = None

df.isna().sum()

id             0
data_tweet     0
texto          0
sentimento     0
texto_norm    12
dtype: int64

In [85]:
df.dropna(subset=['texto_norm'], inplace=True)

In [86]:
print(df.shape)

(5763, 5)


# ToDo 3

Separe a coluna texto_tratado em conjunto de treino e teste na proporção 70/30

In [87]:
df.head()

,id,data_tweet,texto,sentimento,texto_norm
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro,catedral de santo antonio governador valadares mg
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",Neutro,governador valadares minas gerais
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",Neutro,governador valadares minas gerais
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,Negativo,psol vai questionar aumento de vereadores pref...
5,5,Sat Jan 07 13:47:55 +0000 2017,""" bom é bandido morto""\nDeputado Cabo Júlio é ...",Neutro,bom bandido morto deputado cabo julio condenad...


In [88]:
X = df['texto_norm']
y = df['sentimento']

In [89]:
from sklearn.model_selection import train_test_split

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [91]:
print(X_train.shape)
print(X_test.shape)

(4034,)
(1729,)


# ToDo 4

Transforme os dados para criar a representação numérica dos textos. Use uma versão com CountVectorizer e outra com TFIDFVectorizer

## BoW

In [92]:
from sklearn.feature_extraction.text import CountVectorizer

In [93]:
# Train Set
vect = CountVectorizer(ngram_range=(1,1), lowercase=False)
vect.fit(X_train)
X_train_bow = vect.transform(X_train)
X_train_bow

<4034x5539 sparse matrix of type '<class 'numpy.int64'>'
	with 47392 stored elements in Compressed Sparse Row format>

In [94]:
# Test Set
X_test_bow = vect.transform(X_test)
X_test_bow

<1729x5539 sparse matrix of type '<class 'numpy.int64'>'
	with 19040 stored elements in Compressed Sparse Row format>

In [102]:
# Representação BoW
print(X_train_bow[:1])

  (0, 1360)	2
  (0, 1642)	1
  (0, 1666)	1
  (0, 1739)	1
  (0, 2539)	1
  (0, 2830)	1
  (0, 3313)	1
  (0, 3569)	1
  (0, 4072)	1
  (0, 4179)	1
  (0, 4728)	1
  (0, 5175)	1


## TF-IDF

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [96]:
# Train Set
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
vect.fit(X_train)
X_train_tfidf = vect.transform(X_train)
X_train_tfidf

<4034x5539 sparse matrix of type '<class 'numpy.float64'>'
	with 47392 stored elements in Compressed Sparse Row format>

In [97]:
# Test Set
X_test_tfidf = vect.transform(X_test)
X_test_tfidf

<1729x5539 sparse matrix of type '<class 'numpy.float64'>'
	with 19040 stored elements in Compressed Sparse Row format>

In [101]:
# Representação TF-IDF
print(X_train_tfidf[:1])

  (0, 5175)	0.2629330229264354
  (0, 4727)	0.2678863366110196
  (0, 4178)	0.2783295967143665
  (0, 4071)	0.25477993961651946
  (0, 3568)	0.4269274910984725
  (0, 3312)	0.16380757215655944
  (0, 2829)	0.4601614616161445
  (0, 2538)	0.35735858271161447
  (0, 1738)	0.12469597980884331
  (0, 1665)	0.204676106532364
  (0, 1641)	0.2686589766811486
  (0, 1359)	0.19634200164711335


# ToDo5

Treine uma árvore de decisão nas duas abordagens e compare seus resultados

In [111]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

## BoW

In [109]:
print(X_train_bow[:1])

  (0, 1360)	2
  (0, 1642)	1
  (0, 1666)	1
  (0, 1739)	1
  (0, 2539)	1
  (0, 2830)	1
  (0, 3313)	1
  (0, 3569)	1
  (0, 4072)	1
  (0, 4179)	1
  (0, 4728)	1
  (0, 5175)	1


In [106]:
dt_classifier = DecisionTreeClassifier()

# Define a parameter grid for GridSearchCV
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(dt_classifier, param_grid, cv=5)
grid_search.fit(X_train_bow, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}


c:\Users\Jackson Simionato\.conda\envs\pyspark_env\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
360 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Jackson Simionato\.conda\envs\pyspark_env\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Jackson Simionato\.conda\envs\pyspark_env\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\Jackson Simionato\.conda\envs\pyspark_env\lib\site-packages\sklearn\base.py", line 638,

In [113]:
dt_classifier_bow = DecisionTreeClassifier(**best_params)

dt_classifier_bow.fit(X_train_bow, y_train)

y_pred = dt_classifier_bow.predict(X_test_bow)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy - BoW:", accuracy)

Accuracy - BoW: 0.9034123770965876


## TF-IDF

In [114]:
dt_classifier = DecisionTreeClassifier()

# Define a parameter grid for GridSearchCV
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(dt_classifier, param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}


c:\Users\Jackson Simionato\.conda\envs\pyspark_env\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
360 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Jackson Simionato\.conda\envs\pyspark_env\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Jackson Simionato\.conda\envs\pyspark_env\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\Jackson Simionato\.conda\envs\pyspark_env\lib\site-packages\sklearn\base.py", line 638,

In [149]:
dt_classifier_tfidf = DecisionTreeClassifier(max_depth=20, min_samples_split=2, random_state=42)

dt_classifier_tfidf.fit(X_train_tfidf, y_train)

y_pred = dt_classifier_tfidf.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy - TFIDF:", accuracy)

Accuracy - TFIDF: 0.9161364950838635


# ToDo 6

Crie uma função que lematiza as palavras da coluna texto_tratado apenas se elas forem um verbo. Depois, crie uma nova coluna chamada texto_tratado_lemma que conterá o resultado da aplicação da função na coluna texto_tratado.

Dica: use o Corpus pt_core_news_sm como referência para determinar a classe gramatical da palavra

In [168]:
import spacy
nlp = spacy.load('pt_core_news_sm')

In [181]:
def lemmatize_verbs(text):
    text = nlp(text)
    text_lemmatized = [token.lemma_ if token.pos_ == 'VERB' else token.orth_ for token in text]
    return ' '.join([t for t in text_lemmatized])

In [184]:
df['texto_norm_lemma'] = df['texto_norm'].apply(lemmatize_verbs)

In [185]:
df

,id,data_tweet,texto,sentimento,texto_norm,texto_norm_lemma
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro,catedral de santo antonio governador valadares mg,catedral de santo antonio governador valadares mg
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",Neutro,governador valadares minas gerais,governador valadares minas gerais
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",Neutro,governador valadares minas gerais,governador valadares minas gerais
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,Negativo,psol vai questionar aumento de vereadores pref...,psol vai questionar aumento de vereadores pref...
5,5,Sat Jan 07 13:47:55 +0000 2017,""" bom é bandido morto""\nDeputado Cabo Júlio é ...",Neutro,bom bandido morto deputado cabo julio condenad...,bom bandido morto deputado cabo julio condenar...
...,...,...,...,...,...,...
5770,8194,Thu Feb 09 11:48:07 +0000 2017,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo,trio preso suspeito de roubo trafico abuso sex...,trio prender suspeito de roubo trafico abuso s...
5771,8195,Thu Feb 09 12:10:19 +0000 2017,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo,trio preso suspeito de roubo trafico abuso sex...,trio prender suspeito de roubo trafico abuso s...
5772,8196,Thu Feb 09 12:04:17 +0000 2017,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo,trio preso suspeito de roubo trafico abuso sex...,trio prender suspeito de roubo trafico abuso s...
5773,8197,Thu Feb 09 12:10:04 +0000 2017,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo,trio preso suspeito de roubo trafico abuso sex...,trio prender suspeito de roubo trafico abuso s...


In [165]:
tokens = [(token.orth_, token.pos_) for token in texto]
tokens

[('Meu', 'DET'),
 ('texto', 'NOUN'),
 ('gostaria', 'VERB'),
 ('de', 'SCONJ'),
 ('ser', 'AUX'),
 ('melhor', 'ADJ'),
 ('escrito', 'VERB')]

In [170]:
verbs = [token.lemma_ if token.pos_ == 'VERB' else token.orth_ for token in texto]
' '.join(verbs)

'Meu texto gostar de ser melhor escrever'

# ToDo 7

repita os ToDo 3, ToDo 4 e ToDo 5, usando como feature a coluna texto_tratado_lemma, e veja se os resultados tiveram melhora.

In [186]:
X = df['texto_norm_lemma']
y = df['sentimento']

In [187]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## BoW

In [188]:
# Train Set
vect = CountVectorizer(ngram_range=(1,1), lowercase=False)
vect.fit(X_train)
X_train_bow = vect.transform(X_train)
X_train_bow

<4034x5287 sparse matrix of type '<class 'numpy.int64'>'
	with 47405 stored elements in Compressed Sparse Row format>

In [189]:
# Test Set
X_test_bow = vect.transform(X_test)
X_test_bow

<1729x5287 sparse matrix of type '<class 'numpy.int64'>'
	with 19132 stored elements in Compressed Sparse Row format>

In [190]:
# Representação BoW
print(X_train_bow[:1])

  (0, 1338)	2
  (0, 1599)	1
  (0, 1625)	1
  (0, 1697)	1
  (0, 2442)	1
  (0, 2719)	1
  (0, 3183)	1
  (0, 3431)	1
  (0, 3907)	1
  (0, 3987)	1
  (0, 4512)	1
  (0, 4944)	1


In [193]:
dt_classifier = DecisionTreeClassifier(random_state=42)

# Define a parameter grid for GridSearchCV
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(dt_classifier, param_grid, cv=5)
grid_search.fit(X_train_bow, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}


c:\Users\Jackson Simionato\.conda\envs\pyspark_env\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
360 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Jackson Simionato\.conda\envs\pyspark_env\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Jackson Simionato\.conda\envs\pyspark_env\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\Jackson Simionato\.conda\envs\pyspark_env\lib\site-packages\sklearn\base.py", line 638,

In [205]:
dt_classifier_bow = DecisionTreeClassifier(max_depth=30, min_samples_split=2, random_state=42)

dt_classifier_bow.fit(X_train_bow, y_train)

y_pred = dt_classifier_bow.predict(X_test_bow)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy - BoW:", accuracy)

Accuracy - BoW: 0.9224985540775015


## TF-IDF

In [206]:
# Train Set
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
vect.fit(X_train)
X_train_tfidf = vect.transform(X_train)
X_train_tfidf

<4034x5268 sparse matrix of type '<class 'numpy.float64'>'
	with 47401 stored elements in Compressed Sparse Row format>

In [207]:
# Test Set
X_test_tfidf = vect.transform(X_test)
X_test_tfidf

<1729x5268 sparse matrix of type '<class 'numpy.float64'>'
	with 19135 stored elements in Compressed Sparse Row format>

In [213]:
dt_classifier_tfidf = DecisionTreeClassifier(max_depth=20, random_state=42)

dt_classifier_tfidf.fit(X_train_tfidf, y_train)

y_pred = dt_classifier_tfidf.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy - TFIDF:", accuracy)

Accuracy - TFIDF: 0.9161364950838635


# Conclusão

Não houve grande mudança após a lemmatização

Além disso, o desempenho dos dois modelos foi bastante semelhante, com boa qualidade. Entretanto, percebe-se que o conjunto de treino e teste não é de boa qualidade. Existem diversos tweets praticamente identicos escritos em momentos diferentes, o que pode estar facilitando o treinamento e classificação do modelo.